In [370]:
#imporn nessasary packages
import pandas as pd
import numpy as np
import re
#from autocorrect import spell
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
from nltk.stem import LancasterStemmer
from nltk.stem import PorterStemmer
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from itertools import combinations

In [362]:
df = pd.read_csv("/Users/FOMIOLNY/desktop/Fedcentral_Assets.csv")
df.head(3)

,ID,Date Uploaded,Date Approved,Display Name,Type of File,Author (name),Author (email),Description,Short Description,File Use,...,Downloadable,Packages,Marketing Materials,Design File,Graphics,Best Practices,Research & reports,Training Materials,Public Tag Names,type
0,https://fedcentral.org/assets/view/375722,10/15/15 20:56,10/15/15 20:56,MetroWest Able Presentation Audio,mp3,Rina Krevat,rina.krevat@jewishfederations.org,Jewish Federation of Greater MetroWest NJMetro...,NaN,full,...,Y,Access Granted Fedovation,N,N,N,N,N,N,disability | 2014 | fedovations | jewish feder...,audio
1,https://fedcentral.org/assets/view/375723,10/15/15 20:56,10/15/15 20:56,RAISE Presentation Audio,mp3,Rina Krevat,rina.krevat@jewishfederations.org,Jewish Federation of Greater OrlandoRAISE (Rec...,NaN,full,...,Y,Access Granted Fedovation,N,N,N,N,N,N,disability | 2014 | jewish federation of great...,audio
2,https://fedcentral.org/assets/view/375801,10/16/15 14:20,10/16/15 14:20,Beyond Happy Hour Full Presentation Audio,mp3,Rina Krevat,rina.krevat@jewishfederations.org,Beyond Happy Hour: Solutions to the Young Adul...,NaN,full,...,Y,Beyond Happy Hour Fedovation,N,N,N,N,N,N,2014 | fedovations | audio,audio


In [363]:
df2 = pd.read_csv("/Users/FOMIOLNY/desktop/post_question.csv")
df2.head(3)

,ID,Type,Title,Answers,Comments,Hi-Fives,Downloads,Score
0,3137423,Question,I have created a group on FedCentral for Commu...,153,0,5,0,903
1,3149073,Question,Have you created a Israel at 70 logo?,22,0,2,0,141
2,3128136,Question,Does your Federation publish a community newsl...,23,0,0,0,140


In [364]:
df['Display Name copy'] = df['Display Name']
df2['Title copy'] = df2['Title']

In [365]:
df['Display Name copy'] = df['Display Name copy'].str.lower()
df2['Title copy'] = df2['Title copy'].str.lower()

In [366]:
df['Display Name copy'] = df['Display Name copy'].str.replace(r'(?!@)[^\w\s]','')
df2['Title copy'] = df2['Title copy'].str.replace(r'(?!@)[^\w\s]','')
df2['Title copy'] = df2['Title copy'].str.replace(r'(?!@)[^\w\s]','')

In [367]:
#function that replaces NAs with blanks
def replace_na(df,columns):
    df[columns] = df[columns].fillna('')

def apply_tokenization(df, columns):
    stop_words = set(stopwords.words('english'))
    for column in columns:
        df[column] = df[column].apply(word_tokenize)
        #remove stop words    
        for i in range(0,len(df)):
            df[column][i] =[w for w in df[column][i] if w not in stop_words | {''} ]    

            
columns_list = ['Display Name copy']
replace_na(df, columns_list)
apply_tokenization(df, columns_list)

columns_list2 = ['Title copy']
replace_na(df2, columns_list2)
apply_tokenization(df2, columns_list2)

df2.head(3)

/Users/FOMIOLNY/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,ID,Type,Title,Answers,Comments,Hi-Fives,Downloads,Score,Title copy
0,3137423,Question,I have created a group on FedCentral for Commu...,153,0,5,0,903,"[created, group, fedcentral, community, campai..."
1,3149073,Question,Have you created a Israel at 70 logo?,22,0,2,0,141,"[created, israel, 70, logo]"
2,3128136,Question,Does your Federation publish a community newsl...,23,0,0,0,140,"[federation, publish, community, newsletter, c..."


In [371]:
def convert_lemma(df, columns):
    porter = PorterStemmer()

    for column in columns:
        for i in range(0,len(df)):
            df[column][i] =[porter.stem(w) for w in df[column][i]]    

convert_lemma(df, columns_list)
df[columns_list].head(3)

convert_lemma(df2, columns_list2)
df2[columns_list2].head(3)

/Users/FOMIOLNY/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Title copy
0,"[creat, group, fedcentr, commun, campaign, pro..."
1,"[creat, israel, 70, logo]"
2,"[feder, publish, commun, newslett, call, launc..."


In [372]:
for index, qp in df2.iterrows():
    print(qp['Title'])    
    print('')
    print('assets:')
    for index,asset in df.iterrows():
        if set(asset['Display Name copy']).issubset(set(qp['Title copy']))==True:
            print('        '+ str(asset['Display Name']))
    print(' ')
    print('===============================================================')

I have created a group on FedCentral for Community Campaign Pros - primarily focusing on strategies for under $1000 (direct mail, telegiving, ephilanthropy etc.) Let me know if you want to join!

assets:
 
Have you created a Israel at 70 logo?

assets:
 
Does your Federation publish a community newsletter? If so, what is it called? We are launching a monthly e-newsletter with news from our partners and are searching for a name that's Jewish but not too Jewish.

assets:
 
Is this a draconian contract from our Graphic Designer?

assets:
 
Teen Israel Scholarship $
Does your federation provide any $ towards teen Israel travel? What relationship, if any, does the program have to local youth groups and/or synagogues? Please share URL of your program's online presence in answer, thanks!

assets:
 
Can you please share your 2017 Film Festival films?

assets:
 
Gifts at Women's Philanthropy LOJ?

assets:
 
How is your Luminate transition going?

assets:
 
Does anyone have donor advised funds w

 
Major Women's Event Speaker: Who was a draw for attendance? What case for giving speakers have made the largest impact on campaign? 

assets:
 
I'm looking to give women who receive a Lion or Pom pin a card to accompany it--one that shares more about its background and encourages them to wear it often and proudly. Does anyone have an example of text that they use for something like this?

assets:
 
We are doing a video of individual recipient impact stories (not agencies) Anyone have examples of these types of videos or scripts for them? We will be getting recipients of PJ, Young Adult, JCRC security, Holocaust survivors, camp, Israel Experience.

assets:
 
We are considering getting a handicap accessible van for our small community, has anyone else done this?

assets:
 
Do any Federations send thank you letters when donors pay off their campaign pledges?

assets:
 
What adorns your entryway? 

Check this out!

assets:
 
Have anyone been involved in using Amazon Smile?

assets:
 
We 

 
Chicago's YLD Big Event Fundraiser marketing materials

assets:
        Marketing for Fundraising Event
 
The JCCA is currently looking into hosting a one or two-day Facility Manager/Director's Conference. I know some of us own/operate campuses.  Would this be something that Federations would be interested in collaborating on?

assets:
 
What is your Federation's kashrut policy?

assets:
 
Looking for a professional Israeli dance troupe, other than Bat Sheva, to bring here for Israel @ 70 next year. Any suggestions?

assets:
 
to those of you who experienced bomb threats today, are you seeing an unsual media interest this time.  The first time this happened, we had no media inquiries.  Today, we have been called by all four network television channels, NPR radio, and others.

assets:
 
Does the Federation staff in your community get a professional courtesy discount with the JCC?

Thank you!
Sheila

assets:
 
[Updated Resources] Federations Working to Promote Religious Pluralism in Is

 
Recommendations for freelance graphic designers

assets:
 
I was recently hired as a Director of Impact Assessment & Learning – basically an in-house evaluation and impact management role. Do any of your federations have a similar position? Would love to connect with any counterparts!

assets:
 
Interactive Rosh Hashanah App from Toronto

assets:
 
We are looking to bring in Jane Weitzman for our Lion Pom event.  Has anyone had her in for a campaign event?  If so, what was the feedback?  Is her message more about philanthropy in general or does she have a specific message and tie to Federation?

assets:
 
Who is the signer of your acknowledgement letters? Lay leader, CEO, development staff? Thanks! 

Also, can you share any new language you are using for 2018? Our letter needs a rewrite!

assets:
        LEADERS
 
Does anyone have an application for security grants? We are considering funding security grants to our community. Can you share the grant guidelines, process and grant leve

 
Good morning, 
We are looking at some new software to replace a legacy system. I would like to know what donor management system you are using on the front-end and what accounting system is being used on the back-end. 
Thanks
Les

assets:
 
Has anyone had Erica Brown speak at a big event?

assets:
 
What is the breakout of how you advertise/promote your organization? Direct mail, Facebook/social media, Print (what type of publications), other? And how are you targeting prospective donors?

assets:
 
Indiana University: Lilly Family School of Philanthropy Resources

assets:
 
Does anyone use a renewal process for donors under $100? ie, thanks for your past gift, if we don't hear from you we will renew you at the same level as last year?

assets:
 
How does your community honor Jewish Educators? Do you have a community wide celebration or award program? Does each individual school have their own recognition program?

assets:
 
Can anyone recommend a speaker for a Planned giving day?  O

...

 
IDF Killed in Car Ramming | Does anyone plan to release a statement?

assets:
 
Does your community have a minimum gift requirement for your women's division steering committee/board?

assets:
 
Has any Federation had Mehereta Baruch speak at an event? If so, can we speak or email with you to hear about feedback?

assets:
 
Countdown to the 6 Day War Project - Video #7

assets:
 
group trip to Israel 

assets:
 
REMINDER: I&O Webinar THIS WEEK

assets:
 
DOL Overtime Rules: What Now?

assets:
 
Israel and Overseas at GA 2016

assets:
 
Hello! Does anyone have any language around our partnership with JFS? We are doing a tribute ad for their gala and want to do something more than the standard "mazel tov to the honorees". Thanks! 

assets:
 
Speaker Evaluation: Nanette Fridman

assets:
 
Why Are Israeli Women More Generous Than Their Male Counterparts?

assets:
 
How do you define- in an objective manner- a quality gift?

assets:
 
Legendary entertainer Dudu Fisher launches a rare worl

 
JFNA Press Release: $25M in Federal Funding to Bolster Nonprofit Security

assets:
 
[REMINDER] 5/16 How Do We Connect With Israel?

assets:
 
Jewish Education & Engagement Webinar Reminder! Monday, 9/18: Who Are The Jews In Our Pews? Register Today!

assets:
        Jewish Education & Engagement Webinar
 
Has anyone brought in Dr. Scott Gottlieb to speak in your community?  Maimonides Society or otherwise.  Thanks!

assets:
 
Additional Update from Israel (May 6, 2pm ISR)

assets:
 
Has anyone shown the film Women in Sink?

assets:
 
In the ever increasing world of designated rather than federated giving, what are people looking for in demographics/criteria/skill sets for the make up of their Boards of Directors?

assets:
        Designated Giving 
 
Avi Jorisch

assets:
 
TODAY! (6/20) JFNA Webinar: Investing in the Arab Israeli Community

assets:
 
iRep Pluralism Tracker (September 2017)

assets:
 
Giving Tuesday

assets:
 
Director of Community Relations - Jewish Federation of Na

 
Can someone provide me with some roles/responsibilities for a Women's Philanthropy Committee? We're also looking for information on term limits for leadership.

assets:
 
Seeking job description sample: Combo administrator and bookkeeper position. Or, alternatively, one of each. Thank you!

assets:
 
Federation Funded MasterClasses

assets:
 
DAFs: IRS "rules" on Bifurcation and Pledges

assets:
 
JFNA NYL Shabbaton in LA (Nov 10-12) before the GA

assets:
 
How Do We Build Jewish Families? Jewish Education & Engagement Webinar TOMORROW January 25th

assets:
        Jewish Education & Engagement Webinar
 
Who Are Israeli Philanthropists?

assets:
 
Do you know a soon-to-be college student?

assets:
 
JFNA August 2017 Congressional Recess Memo with Talking Points and Issues Briefs

assets:
 
Speaker Evaluation: Nanette Fridman

assets:
 
Video of Jerusalem's Complexity and Beauty

assets:
        Jerusalem
        Jerusalem 
 
Interested in knowing what B&P speakers your Federation ha

 
What are different federations policies/guidelines around family giving specifically as it relates to BGS? How are couples handled that are married vs engaged vs living together? Thanks!

assets:
 
Post-election wisdom from Noa Kushner

assets:
 
New Pew analysis: U.S. Jews more likely than Christians to say Trump favors Israelis too much

assets:
 
Capital grants for Security Infrastructure

assets:
 
Violent Escalation in Israel (Saturday May 6)

assets:
 
Our Young Men's Group is holding a retreat that is being co-facilitated by former Men's Group chairs. Do you  have suggestions for high quality leadership development and community engagement programming for young leaders? Thanks!

assets:
        LEADERS
        Leadership Development Development Programs
 
Has anyone heard of the Israel Public Diplomacy Forum and Eytan Gilboa? Did you have him speak in your area and what did you think?

assets:
 
Federation/Jewish Free Loan Society relationships

assets:
 
Award of excellence f

 
Did your community focus your campaign around the Beatles?

assets:
 
Do you have a specific policy for how to handle a bequest paid over an extended period of time?

assets:
 
UBIT on Transportation Benefits: Delay?

assets:
 
[REMINDER] JFNA Webinar TODAY | Donor Challenges, Communal Solutions

assets:
 
Jewish Agency Invites Aerosmith’s Steven Tyler to Apply for Aliyah

assets:
 
Compelling personal story of overseas speaker available October 29 – November 5 ( Oct 31 & Nov 1 already booked)

assets:
 
Jewish Education & Engagement at Professional Institute! See you in Houston!

assets:
 
A Changing Europe and Jewish Communities

assets:
 
President Trump's will visit Israel next week What could the impact of this trip be?

assets:
 
Yesod program in Shalom New Haven

assets:
 
Register now for FedEngage: Federations Focus on Engagement

assets:
 
IMPT: Shimon Peres suffers stroke

assets:
 
Countdown to the 6 Day War Project - Video #9

assets:
 
Has anyone brought Dr. Miriam Hara